# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 3 2023 11:43AM,258633,10,37791,"Digital Brands, LLC",Released
1,Mar 3 2023 11:43AM,258633,10,37792,"Digital Brands, LLC",Released
2,Mar 3 2023 11:08AM,258630,19,ADV80013151,"AdvaGen Pharma, Ltd",Executing
3,Mar 3 2023 11:08AM,258630,19,ADV80013152,"AdvaGen Pharma, Ltd",Executing
4,Mar 3 2023 11:08AM,258630,19,ADV80013153,"AdvaGen Pharma, Ltd",Executing
5,Mar 3 2023 11:08AM,258630,19,ADV80013154,"AdvaGen Pharma, Ltd",Executing
6,Mar 3 2023 11:08AM,258630,19,ADV80013155,"AdvaGen Pharma, Ltd",Released
7,Mar 3 2023 11:08AM,258630,19,ADV80013156,"AdvaGen Pharma, Ltd",Released
8,Mar 3 2023 11:08AM,258630,19,ADV80013157,"AdvaGen Pharma, Ltd",Released
9,Mar 3 2023 11:08AM,258630,19,ADV80013158,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,258627,Released,1
22,258628,Executing,1
23,258630,Executing,4
24,258630,Released,7
25,258633,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258626,NaN,1.0,NaN
258627,NaN,NaN,1.0
258628,NaN,1.0,NaN
258630,NaN,4.0,7.0
258633,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258556,0.0,9.0,2.0
258557,0.0,0.0,1.0
258562,0.0,0.0,1.0
258578,0.0,1.0,0.0
258598,9.0,22.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258556,0,9,2
258557,0,0,1
258562,0,0,1
258578,0,1,0
258598,9,22,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258556,0,9,2
1,258557,0,0,1
2,258562,0,0,1
3,258578,0,1,0
4,258598,9,22,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258556,,9,2
1,258557,,,1
2,258562,,,1
3,258578,,1,
4,258598,9,22,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 3 2023 11:43AM,258633,10,"Digital Brands, LLC"
2,Mar 3 2023 11:08AM,258630,19,"AdvaGen Pharma, Ltd"
13,Mar 3 2023 11:06AM,258628,12,Hush Hush
14,Mar 3 2023 10:55AM,258627,20,"Andelyn Biosciences, Inc."
15,Mar 3 2023 10:47AM,258626,16,"SHL Pharma, LLC"
16,Mar 3 2023 10:15AM,258624,19,"NAPP Technologies, LLC"
18,Mar 3 2023 10:14AM,258623,19,"NAPP Technologies, LLC"
19,Mar 3 2023 10:06AM,258622,10,Bio-PRF
30,Mar 3 2023 9:57AM,258621,19,"GCH Granules USA, Inc."
31,Mar 3 2023 9:45AM,258618,19,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 3 2023 11:43AM,258633,10,"Digital Brands, LLC",,,2
1,Mar 3 2023 11:08AM,258630,19,"AdvaGen Pharma, Ltd",,4,7
2,Mar 3 2023 11:06AM,258628,12,Hush Hush,,1,
3,Mar 3 2023 10:55AM,258627,20,"Andelyn Biosciences, Inc.",,,1
4,Mar 3 2023 10:47AM,258626,16,"SHL Pharma, LLC",,1,
5,Mar 3 2023 10:15AM,258624,19,"NAPP Technologies, LLC",,,2
6,Mar 3 2023 10:14AM,258623,19,"NAPP Technologies, LLC",,,1
7,Mar 3 2023 10:06AM,258622,10,Bio-PRF,,3,8
8,Mar 3 2023 9:57AM,258621,19,"GCH Granules USA, Inc.",,1,
9,Mar 3 2023 9:45AM,258618,19,"Emersa Waterbox, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 3 2023 11:43AM,258633,10,"Digital Brands, LLC",2,,
1,Mar 3 2023 11:08AM,258630,19,"AdvaGen Pharma, Ltd",7,4,
2,Mar 3 2023 11:06AM,258628,12,Hush Hush,,1,
3,Mar 3 2023 10:55AM,258627,20,"Andelyn Biosciences, Inc.",1,,
4,Mar 3 2023 10:47AM,258626,16,"SHL Pharma, LLC",,1,
5,Mar 3 2023 10:15AM,258624,19,"NAPP Technologies, LLC",2,,
6,Mar 3 2023 10:14AM,258623,19,"NAPP Technologies, LLC",1,,
7,Mar 3 2023 10:06AM,258622,10,Bio-PRF,8,3,
8,Mar 3 2023 9:57AM,258621,19,"GCH Granules USA, Inc.",,1,
9,Mar 3 2023 9:45AM,258618,19,"Emersa Waterbox, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 3 2023 11:43AM,258633,10,"Digital Brands, LLC",2,,
1,Mar 3 2023 11:08AM,258630,19,"AdvaGen Pharma, Ltd",7,4,
2,Mar 3 2023 11:06AM,258628,12,Hush Hush,,1,
3,Mar 3 2023 10:55AM,258627,20,"Andelyn Biosciences, Inc.",1,,
4,Mar 3 2023 10:47AM,258626,16,"SHL Pharma, LLC",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 3 2023 11:43AM,258633,10,"Digital Brands, LLC",2.0,NaN,NaN
1,Mar 3 2023 11:08AM,258630,19,"AdvaGen Pharma, Ltd",7.0,4.0,NaN
2,Mar 3 2023 11:06AM,258628,12,Hush Hush,NaN,1.0,NaN
3,Mar 3 2023 10:55AM,258627,20,"Andelyn Biosciences, Inc.",1.0,NaN,NaN
4,Mar 3 2023 10:47AM,258626,16,"SHL Pharma, LLC",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 3 2023 11:43AM,258633,10,"Digital Brands, LLC",2.0,0.0,0.0
1,Mar 3 2023 11:08AM,258630,19,"AdvaGen Pharma, Ltd",7.0,4.0,0.0
2,Mar 3 2023 11:06AM,258628,12,Hush Hush,0.0,1.0,0.0
3,Mar 3 2023 10:55AM,258627,20,"Andelyn Biosciences, Inc.",1.0,0.0,0.0
4,Mar 3 2023 10:47AM,258626,16,"SHL Pharma, LLC",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1810175,29.0,4.0,0.0
12,517227,0.0,2.0,0.0
15,775785,10.0,9.0,0.0
16,258626,0.0,1.0,0.0
19,1810331,18.0,27.0,9.0
20,258627,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1810175,29.0,4.0,0.0
1,12,517227,0.0,2.0,0.0
2,15,775785,10.0,9.0,0.0
3,16,258626,0.0,1.0,0.0
4,19,1810331,18.0,27.0,9.0
5,20,258627,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,29.0,4.0,0.0
1,12,0.0,2.0,0.0
2,15,10.0,9.0,0.0
3,16,0.0,1.0,0.0
4,19,18.0,27.0,9.0
5,20,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,29.0
1,12,Released,0.0
2,15,Released,10.0
3,16,Released,0.0
4,19,Released,18.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Completed,0.0,0.0,0.0,0.0,9.0,0.0
Executing,4.0,2.0,9.0,1.0,27.0,0.0
Released,29.0,0.0,10.0,0.0,18.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Completed,0.0,0.0,0.0,0.0,9.0,0.0
1,Executing,4.0,2.0,9.0,1.0,27.0,0.0
2,Released,29.0,0.0,10.0,0.0,18.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Completed,0.0,0.0,0.0,0.0,9.0,0.0
1,Executing,4.0,2.0,9.0,1.0,27.0,0.0
2,Released,29.0,0.0,10.0,0.0,18.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()